In [1]:
import numpy as np
import pinocchio as pin

In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import z1_mj_description, z1_description, skydio_x2_mj_description, skydio_x2_description

In [3]:
ROBOT_URDF = skydio_x2_description.URDF_PATH
ROBOT_URDF = z1_description.URDF_PATH

In [4]:
model = pin.buildModelFromUrdf(ROBOT_URDF)
data = model.createData()


In [5]:
q = np.random.randn(model.nq)
if model.nq != model.nv:
    q[3:7]/=np.linalg.norm(q[3:7])
dq = np.random.randn(model.nv)
ddq  = np.random.randn(model.nv)
data.q = q.copy()
data.dq = dq.copy()
data.ddq = ddq.copy()


### Custom Regressor

In [6]:
from time import perf_counter

In [17]:
pin.computeAllTerms(model, data, q, dq)
pin.forwardKinematics(model, data, q, dq, ddq)

def Y_body(v, a_g):
    a1, a2, a3, a4, a5, a6 = a_g
    v1, v2, v3, v4, v5, v6 = v

    Y = np.array([
    [a1 - v2*v6 + v3*v5, -v5**2 - v6**2, -a6 + v4*v5, a5 + v4*v6, 0, 0, 0, 0, 0, 0],
    [a2 + v1*v6 - v3*v4, a6 + v4*v5, -v4**2 - v6**2, -a4 + v5*v6, 0, 0, 0, 0, 0, 0],
    [a3 - v1*v5 + v2*v4, -a5 + v4*v6, a4 + v5*v6, -v4**2 - v5**2, 0, 0, 0, 0, 0, 0],
    [0, 0, a3 - v1*v5 + v2*v4, -a2 - v1*v6 + v3*v4, a4, a5 - v4*v6, -v5*v6, a6 + v4*v5, v5**2 - v6**2, v5*v6],
    [0, -a3 + v1*v5 - v2*v4, 0, a1 - v2*v6 + v3*v5, v4*v6, a4 + v5*v6, a5, -v4**2 + v6**2, a6 - v4*v5, -v4*v6],
    [0, a2 + v1*v6 - v3*v4, -a1 + v2*v6 - v3*v5, 0, -v4*v5, v4**2 - v5**2, v4*v5, a4 - v5*v6, a5 + v4*v6, a6]
])

    return Y

def bodyRegressor(ind):
    """Body regressor
    Derivation of body regressor is accordingly to the paper: https://arxiv.org/pdf/1610.08703.pdf
    link: https://colab.research.google.com/drive/1xFte2FT0nQ0ePs02BoOx4CmLLw5U-OUZ#scrollTo=FfkIBpi1wLXz
    """
    vel = pin.getVelocity(model,
                    data,
                    ind,
                    pin.LOCAL) 
    
    accel = pin.getAcceleration(model,
                    data,
                    ind,
                    pin.LOCAL)  
    
    rotation = data.oMi[ind].rotation

    v = np.array(vel)
    a = np.array(accel)
    a_g = a 
    a_g[:3] +=  rotation.T @ np.array([0,0,9.81])




    return Y_body(v, a_g)

njoints = model.njoints - 1
force_Y = np.zeros((6*njoints, njoints*10))
col_jac = np.zeros((6*njoints, model.nv))

t1 = perf_counter()
for i in range(njoints):
    
    jacobian = pin.getJointJacobian(model, 
                                    data, 
                                    i+1, 
                                    pin.LOCAL)
    col_jac[6*i:6*(i+1),:] = jacobian  
    force_Y[6*i:6*(i+1),10*i:10*(i+1)] = bodyRegressor(i+1) #regressor_body(i+1)
    
Ycustom = col_jac.T @force_Y
t2 = perf_counter()
print(1000*(t2-t1))

0.1952880047610961


In [8]:
Ypin = pin.computeJointTorqueRegressor(model,  data, q, dq, ddq)
# Ypin

In [9]:
np.allclose(Ycustom,Ypin)

True